In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
import pandas as pd

# Read the CSV file into pandas DataFrames
input_data = pd.read_csv('input_sentences.csv')
target_data = pd.read_csv('target_sentences.csv')

# Assuming your CSV files have columns named 'input_sentence' and 'target_sentence'
input_sentences = input_data['input_sentence'].tolist()
target_sentences = target_data['target_sentence'].tolist()


NameError: name 'asl_gloss_tokenizer' is not defined

In [10]:
# Sample English sentences and their ASL gloss translations
# english_sentences = [
#     "hello how are you",
#     "what is your name",
#     "my name is John"
# ]

# asl_gloss_sentences = [
#     "HELLO HOW YOU",
#     "WHAT YOUR NAME",
#     "MY NAME JOHN"
# ]

# Tokenize English sentences
english_tokenizer = Tokenizer(filters='')
english_tokenizer.fit_on_texts(input_sentences)
english_vocab_size = len(english_tokenizer.word_index) + 1

# Tokenize ASL gloss sentences
asl_gloss_tokenizer = Tokenizer(filters='')
asl_gloss_tokenizer.fit_on_texts(target_sentences)
asl_gloss_vocab_size = len(asl_gloss_tokenizer.word_index) + 1

# Convert text sequences to integer sequences
english_sequences = english_tokenizer.texts_to_sequences(input_sentences)
asl_gloss_sequences = asl_gloss_tokenizer.texts_to_sequences(target_sentences)

# Pad sequences to ensure uniform length
max_length = max(len(seq) for seq in english_sequences)
english_sequences_padded = pad_sequences(english_sequences, maxlen=max_length, padding='post')
asl_gloss_sequences_padded = pad_sequences(asl_gloss_sequences, maxlen=max_length, padding='post')


In [8]:
asl_gloss_tokenizer.word_index.keys()

dict_keys(['name', 'hello', 'how', 'you', 'what', 'your', 'my', 'john'])

In [11]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(english_sequences_padded, asl_gloss_sequences_padded, test_size=0.2)

# Define the encoder-decoder model
embedding_dim = 16

encoder_inputs = tf.keras.layers.Input(shape=(max_length,))
encoder_embedding = tf.keras.layers.Embedding(english_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(max_length,))
decoder_embedding = tf.keras.layers.Embedding(asl_gloss_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(asl_gloss_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [12]:
# Compile the model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Train the model
model.fit([X_train, X_train], np.expand_dims(y_train, axis=-1), epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 193ms/step - accuracy: 0.4051 - loss: 5.7931 - val_accuracy: 0.7701 - val_loss: 5.5913
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7635 - loss: 5.2183 - val_accuracy: 0.7701 - val_loss: 2.2953
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7676 - loss: 1.8635 - val_accuracy: 0.7701 - val_loss: 1.6592
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7764 - loss: 1.5265 - val_accuracy: 0.7701 - val_loss: 1.5055
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7770 - loss: 1.4021 - val_accuracy: 0.7701 - val_loss: 1.5046
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7660 - loss: 1.3994 - val_accuracy: 0.7701 - val_loss: 1.5268
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7686 - loss: 1.4090 - val_accuracy: 0.7739 - val_loss: 1.5371
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7840 - loss: 1.3059 - val_accuracy: 0.7759 - val_loss

In [14]:
# Evaluate the model
loss, accuracy = model.evaluate([X_test, X_test], np.expand_dims(y_test, axis=-1))
print("Test Accuracy:", accuracy)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7694 - loss: 2.2519
Test Accuracy: 0.7669753432273865


In [19]:
path = 'encoddecod/eng2gloss.h5'
model.save(path)

In [14]:
max_length_src = max(len(seq) for seq in english_sequences_padded)

input_sentence = "what is your name"

# Tokenize input sentence
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
input_sequence_padded = pad_sequences(input_sequence, maxlen=max_length_src, padding='post')

# Initialize decoder input sequence with start token
decoder_input = np.array([[asl_gloss_tokenizer.word_index['<start>']]])


# Encode the input sequence
encoder_input = input_sequence_padded
reshaped_encoder_input = tf.reshape(encoder_input, (batch_size, 1, encoder_input.shape[-1]))
encoder_output, state_h_enc, state_c_enc = encoder_lstm(reshaped_encoder_input)
encoder_states = [state_h_enc, state_c_enc]



# Generate ASL gloss sequence
decoded_sentence = ''
while True:
    decoder_output, state_h_dec, state_c_dec = decoder_lstm(tf.expand_dims(decoder_input, axis=-1), initial_state=encoder_states)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_prediction = decoder_dense(decoder_output)
    predicted_word_index = tf.argmax(decoder_prediction, axis=-1).numpy()[0][0]
    predicted_word = asl_gloss_tokenizer.index_word[predicted_word_index]
    if predicted_word == '<end>' or len(decoded_sentence.split()) > max_length_tar:
        break
    decoded_sentence += predicted_word + ' '
    # Update decoder input for the next iteration
    decoder_input = np.array([[predicted_word_index]])

print("Input Sentence:", input_sentence)
print("Predicted ASL Gloss:", decoded_sentence)

KeyError: '<start>'

In [9]:
max_length_src = max(len(seq) for seq in english_sequences_padded)
max_num_words=4

input_sentence = "what is your name"

start_token = '<start>'
asl_gloss_tokenizer = Tokenizer(num_words=max_num_words, oov_token='<unk>')  # Adjust parameters as needed
asl_gloss_tokenizer.fit_on_texts([start_token] + asl_gloss_sentences)


# Tokenize input sentence
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
input_sequence_padded = pad_sequences(input_sequence, maxlen=max_length_src, padding='post')

# Initialize decoder input sequence with start token
decoder_input = np.array([[asl_gloss_tokenizer.word_index[start_token]]])


# Encode the input sequence
encoder_input = input_sequence_padded
reshaped_encoder_input = tf.reshape(encoder_input, (batch_size, 1, encoder_input.shape[-1]))
encoder_output, state_h_enc, state_c_enc = encoder_lstm(reshaped_encoder_input)
encoder_states = [state_h_enc, state_c_enc]



# Generate ASL gloss sequence
decoded_sentence = ''
while True:
    decoder_output, state_h_dec, state_c_dec = decoder_lstm(tf.expand_dims(decoder_input, axis=-1), initial_state=encoder_states)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_prediction = decoder_dense(decoder_output)
    predicted_word_index = tf.argmax(decoder_prediction, axis=-1).numpy()[0][0]
    predicted_word = asl_gloss_tokenizer.index_word[predicted_word_index]
    if predicted_word == '<end>' or len(decoded_sentence.split()) > max_length_tar:
        break
    decoded_sentence += predicted_word + ' '
    # Update decoder input for the next iteration
    decoder_input = np.array([[predicted_word_index]])

print("Input Sentence:", input_sentence)
print("Predicted ASL Gloss:", decoded_sentence)

KeyError: '<hello>'

In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the saved model
# model = load_model('path/to/saved/model.h5')

# Define English and ASL gloss tokenizers
# Replace these with your actual tokenizer instances


# Define a function to translate English sentence to ASL gloss text
def translate_to_asl(english_sentence):
    # Tokenize input sentence
    input_sequence = english_tokenizer.texts_to_sequences([english_sentence])
    
    # Pad sequences to ensure uniform length
    max_length_src = max(len(seq) for seq in english_sequences_padded)  # Define the maximum sequence length used during training
    input_sequence_padded = pad_sequences(input_sequence, maxlen=max_length_src, padding='post')
    
    # Generate predictions
    predicted_sequence = model.predict(input_sequence_padded,"")
    
    # Convert predicted token indices to ASL gloss text
    asl_gloss_text = ' '.join([asl_gloss_tokenizer.index_word[idx] for idx in predicted_sequence[0]])
    
    return asl_gloss_text

# Test the translation function with an example English sentence
english_sentence = "hello how are you"
predicted_asl_gloss = translate_to_asl(english_sentence)
print("English Sentence:", english_sentence)
print("Predicted ASL Gloss:", predicted_asl_gloss)


ValueError: Layer 'functional_1' expected 2 input(s). Received 1 instead.